## Serving TensorFlow Models (locally)

In this notebook, we'll look at how to serve the Model we created in the previous notebook. This will give us the ability to share a Model with other users.

TensorFlow helpfully includes a "TensorFlow Serving" server, that is specialized to serving TensorFlow models, and optimized for use in production. It's robust, and has the ability to serve more than one model, and also versions of a given model

In order to use it though, we need to have our model saved in a different format. We touched on this at the end of the last notebook, but it works on SavedModel format models.

Luckily, we don't have to lose the portability of HDF5 models, since we can convert between the two formats easily.

After doing that, we'll then serve the model with TF Serving from a docker container, and use a supplied webapp to send a hand(or cursor)-drawn model to the served model and see its predictions

To start, let's load the model from before

In [ ]:
import os
import tensorflow as tf

# Load our saved model
model = tf.keras.models.load_model('mnist-model.h5')

# Save it out in the SavedModel directory format
tf.contrib.saved_model.save_keras_model(model, 'mnist-model')
# Note that at time of writing (TF 1.13), the call is listed as deprecated in TF 2.0
# replaced with tf.keras.experimental.export_saved_model

This will create an output directory, which we can inspect

It contains a subdirectory for this specific version of the model, and inside several files

In [ ]:
!ls -R ./mnist-model

This is the format that TF Serving expects, and is passed as an argument to the container. We can also examine some data about it using the saved_model_cli command

Here we can see that it accepts tensors with a shape (-1, 28, 28, 1). The -1 indicates that the shape is undefined, so we can pass an array of 1 to N tensors with the (28, 28, 1) shape and outputs a single 10-element tensor for each prediction

In [ ]:
!saved_model_cli show --dir "mnist-model/$(ls mnist-model)" --tag_set serve --signature_def serving_default

### Running TF Serving in a container

Now on to setting up TensorFlow Serving

We run it in a docker container with port 8501 exposed. Port 8501 is the HTTP API

In this case, we mount our mnist-model directory as a volume at /models/mnist-model in the container. This simplifies getting the SavedModel into the container. It also supports other storage systems like S3

We also need to pass the name of the model (the directory name of the SavedModel) as an environment variable

In [ ]:
!docker run -d --rm -p 8501:8501 \
    -v "$(pwd)/mnist-model:/models/mnist-model:Z" \
    -e MODEL_NAME=mnist-model \
    tensorflow/serving

### TF Serving's API

TF serving serves a HTTP API, which we can query

There are 3 endpoints that we really care about:
* GET /v1/models/mnist-model - This is where we can see the model is served from. We can call several methods from it
* GET /v1/models/mnist-model/metadata - This shows us metadata about the model
* POST /v1/models/mnist-model:predict - This is what we POST a JSON 4d tensor to in order to get a label for each tensor

Note that simply querying / on the server doesn't tell you much, giving just a 404

In [ ]:
!curl localhost:8501/v1/models/mnist-model

In [ ]:
# As we can see, this tells us about the input and output shapes
!curl localhost:8501/v1/models/mnist-model/metadata

### Using the webapp

Now for the POST, instead of showing it normally, it'd be better to do something nice and interactive. For that purpose, there's an included flask app that we can run in order to show the power of the model interactively.

Running this cell will hang indefinitely while the Flask server runs, you'll need to kill it manually from kernel > interrupt in the menus above

When ready, go to localhost:5000 in the browser (or click the link in the output) to use the app

In [ ]:
!TF_SERVING_URL="http://localhost:8501" python Resources/flask-mnist/flask-mnist.py

### Some tips for the webapp

* The image is not automatically centered, try to center it as much as possible
* The image isn't scaled, you will want to try to scale it to take up most of the canvas